# Packages and Functions

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import dirichlet
from scipy.special import logsumexp, gammaln, digamma, polygamma
from datetime import datetime
import pickle
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
def logdotexp(A, B):
    max_A = np.max(A)
    max_B = np.max(B)
    C = np.dot(np.exp(A - max_A), np.exp(B - max_B))
    np.log(C, out=C)
    C += max_A + max_B
    return C

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load Dataset (Dense)

In [4]:
# path for loading dataset
path = '/content/drive/MyDrive/PhD/Modules/CS5340 Uncertainty Modeling in AI/Project/'
users_ds = np.load(path + 'users_ds_dense.npy', allow_pickle=True)

In [ ]:
# test_ds = [user[-1,:] for user in users_ds]
# users_ds = [user[:-1,:] for user in users_ds]

In [5]:
t_predict = -1                                          # index for holdout period for prediction
test_ds = users_ds[:,t_predict,:]                       # extract holdout period from dataset
test_ds = test_ds[:,np.newaxis,:]                       # align # of dimensions
users_ds = users_ds[:,:t_predict,:]                     # specify training periods

In [6]:
# size of test dataset
test_ds.shape 

(1212, 1, 17768)

In [7]:
# size of training dataset
U = len(users_ds)
T = users_ds[0].shape[0]
N = users_ds[0].shape[1]
U, T, N

(1212, 72, 17768)

In [8]:
# calculate and store total number of ratings per user in a period
users_Nt = np.sum(users_ds, axis=-1)    # number of movies rated by user in each time period
T = np.shape(users_ds)[1]               # final length of time period after trimming
users_Nt.shape 

(1212, 72)

# Multiple runs on same settings

In [9]:
# path for saving the final parameters in .npy format
# path = '/content/drive/MyDrive/PhD/Modules/IS6101 Topics in Machine Learning and Optimization/HMM for CF/Data and Parameters/Parameters/'
# path for excel file to save results in table
# table_path = '/content/drive/MyDrive/PhD/Modules/IS6101 Topics in Machine Learning and Optimization/HMM for CF/Report/Table.xlsx'

# Initial parameters
epsilon = 1e-4          # log likelihood convergence epsilon
K = 5                  # no of latent classes
prior_const = 0.9*K     # affects the parameters of the Dirichlet priors

for r in range(1):
    pi_alpha = prior_const/K * np.ones((K))             # hyperparams for pi
    A_alpha = prior_const/K * np.ones((K,K))            # hyperparams for A
    theta_alpha = prior_const/K * np.ones((K,N))        # hyperparams for theta
    a = np.random.uniform(low=2, high=5, size=(K))      # initialise parameter a randomly
    p = np.random.uniform(low=0.6, high=0.8, size=(K))  # initialise parameter p where p = b / b+1
    b = p / (1-p)                                       # derive parameter b as defined in paper
    pi = dirichlet.rvs(alpha=pi_alpha, size=1)          # initialise pi randomly
    A = np.zeros((K,K)) 
    theta = np.zeros((K,N))
    for k in range(K):
        A[k,:] = dirichlet.rvs(alpha=A_alpha[k,:], size=1)          # initialise matrix A row by row
        theta[k,:] = dirichlet.rvs(alpha=theta_alpha[k,:], size=1)  # initialise theta row by row

    # convert parameters all to log probabilities
    A = np.log(A)
    pi = np.log(pi)
    theta = np.log(theta)

    # EM Algorithm
    start_time = datetime.now() # for keeping track of running time
    a_epsilon = 1e-3            # convergence threshold for optimising parameter a (max 10 iterations)
    old_likelihood = None       # keep track of previous iteration log likelihood 
    iteration = 0               # keep track of number of itrations
    first_part = (gammaln(users_Nt + 1) - gammaln(users_ds + 1).sum(axis=-1))[..., np.newaxis]  # store repeated calculations in multinomial log prob for significant speed up
    while True:
        # E STEP
        
        # log prob of N given z as gamma mixture of poisson i.e. number of articles read
        p_n_ab = gammaln(users_Nt[..., np.newaxis] + a[np.newaxis, np.newaxis, ...]) \
                - gammaln(a)[np.newaxis, np.newaxis, ...] - gammaln(users_Nt+1)[..., np.newaxis] \
                + users_Nt[..., np.newaxis] * np.log(b)[np.newaxis, np.newaxis, ...]  \
                - (users_Nt[..., np.newaxis] + a[np.newaxis, np.newaxis, ...]) * np.log(b+1)[np.newaxis, np.newaxis, ...]

        # log prob of I given z and N as Multinomial(theta) i.e. which articles are read where read=1/unread=0
        second_part = np.dot(users_ds, theta.T)
        p_i_theta = first_part + second_part
        del second_part

        # log prob of joint dist of N, I given z
        p_i_z = p_n_ab + p_i_theta

        # HMM for CF paper definition of alpha and beta
        alpha = np.empty((U,T,K), dtype='float64')
        p_i_i = np.empty((U,T), dtype='float64')

        alpha[:,0] = p_i_z[:,0] + pi
        alpha[:,0] -= logsumexp(alpha[:,0], axis=-1)[...,np.newaxis]
        for t in range(1, T):
            alpha[:,t] = logdotexp(alpha[:,t-1], A) + p_i_z[:,t]
            p_i_i[:,t] = logsumexp(alpha[:,t], axis=-1)
            alpha[:,t] -= p_i_i[:,t][...,np.newaxis]

        beta = np.zeros((U,T,K), dtype='float64')
        for u in range(U):
            for t in range(T-2, -1, -1):
                beta[u,t] = logdotexp(A, (p_i_z[u,t+1] + beta[u,t+1]))
                beta[u,t] -= p_i_i[u,t+1] # normalization

        # numerical issues "divide by zero encountered in log" for the vectorized code below
        # for t in range(T-2, -1, -1):
        #     beta[:,t,:] = logdotexp((p_i_z[:,t+1,:] + beta[:,t+1,:]), A.T)
        #     beta[:,t,:] -= p_i_i[:,t+1][...,np.newaxis] # normalization

        # log prob of Z(t) given I(1:T)
        p_z_i = alpha + beta

        # log prob of Z(t), Z(t+1) given I(1:T)
        p_zz_i = np.empty((U,T-1,K,K), dtype='float64')
        for u in range(U):
            for t in range(T-1):
                p_zz_i[u,t,:,:] = np.tile(alpha[u,t,:], (K,1)).T + A + np.tile(p_i_z[u,t+1,:], (K,1)) + np.tile(beta[u,t+1,:], (K,1)) 
                p_zz_i[u,t,:,:] -= p_i_i[u,t+1][...,np.newaxis,np.newaxis] # normalization

        del alpha
        del beta
        del p_i_i
        del p_i_z

        # CALCULATE EXPECTED LOG LIKELIHOOD
        # check that it is increasing at every iteration and check for convergence condition

        # intial state 
        init = np.sum(np.multiply(np.exp(p_z_i[:,0]), pi[np.newaxis,...]))

        # transitional 
        trans = np.sum(np.multiply(np.exp(p_zz_i), A[np.newaxis,np.newaxis,...]))

        # # of items 
        nbd = np.sum(np.multiply(np.exp(p_z_i), p_n_ab))

        # specific item 
        multi = np.sum(np.multiply(np.exp(p_z_i), p_i_theta))

        if old_likelihood is None:
            old_likelihood = init + trans + nbd + multi
        else:
            new_likelihood = init + trans + nbd + multi 

            if np.isnan(new_likelihood):
                print('Numerical issues in calculation of log likelihood\nPrevious calculated log likelihood = ', old_likelihood)
                break
            if new_likelihood < old_likelihood:
                print('Iteration resulted in lower log likelihood =', new_likelihood)
                break
            if np.abs((new_likelihood - old_likelihood) / old_likelihood) < epsilon:
                old_likelihood = new_likelihood
                print('Iteration', iteration,': log likelihood =', old_likelihood, '\n')
                print('Convergence attained')
                break
            old_likelihood = new_likelihood
        print('Iteration', iteration,': log likelihood =', old_likelihood)            
        iteration += 1

        del p_n_ab
        del p_i_theta

        # M STEP

        # update pi using MAP
        pi_alpha += np.sum(np.exp(p_z_i[:,0]), axis=0)
        if pi_alpha.min() < 1:      # check to ensure no negative probabilities will result
            pi = pi_alpha / np.sum(pi_alpha)
        else:
            pi = (pi_alpha - 1) / (np.sum(pi_alpha) - K)
        pi = np.log(pi)

        # update A using MAP
        A_alpha += np.sum(np.exp(p_zz_i), axis=(0,1))
        if A_alpha.min() < 1:       # check to ensure no negative probabilities will result
            A = A_alpha / np.sum(A_alpha, axis=-1)[...,np.newaxis] # to align the division
        else:
            A = (A_alpha - 1) / (np.sum(A_alpha, axis=-1) - K)[...,np.newaxis] # align the division
        A = np.log(A)

        # update theta using MAP
        # theta_alpha += np.sum(np.multiply(users_ds[:,:,np.newaxis,:], np.exp(p_z_i[...,np.newaxis])), axis=(0,1)) # uses too much memory
        for k in range(K):
            theta_alpha[k,:] += np.sum(np.multiply(users_ds, np.exp(p_z_i[:,:,k][...,np.newaxis])), axis=(0,1))
        if theta_alpha.min() < 1:
            theta = theta_alpha / (np.sum(theta_alpha, axis=-1))[...,np.newaxis]
        else:
            theta = (theta_alpha - 1) / ((np.sum(theta_alpha, axis=-1)) - K)[...,np.newaxis]
        theta = np.log(theta)
        
        # update a using MLE with Newton's method
        for _ in range(10):     
            N_bar = np.sum((np.exp(p_z_i) * users_Nt[...,np.newaxis] + a[np.newaxis,np.newaxis,...]) * 
                        (b / (b + 1))[np.newaxis,np.newaxis,...], axis=(0,1)) / (U*T)
            log_N_bar = np.sum(digamma(np.exp(p_z_i) * users_Nt[...,np.newaxis] + a[np.newaxis,np.newaxis,...]) + 
                            np.log(b / (b + 1))[np.newaxis,np.newaxis,...], axis=(0,1)) / (U*T)
            a_new = (1/a + (log_N_bar - np.log(N_bar) + np.log(a) - digamma(a)) / (a**2 * (1/a - polygamma(1, a))))**-1
            if np.isnan(a).any():
                print('Numerical issues in calculating parameter a')
                break
            if np.sum(np.abs(a_new - a)) / np.sum(a) < a_epsilon:
                a = a_new
                b = N_bar / a  
                break
            a = a_new

        # update b using MLE 
        b = N_bar / a      

    run_time = datetime.now() - start_time
    np.set_printoptions(precision=3)
    print('Execution Time:', run_time)
    print('\npi = ', np.exp(pi),'\n\nA = ', np.exp(A),'\n\na = ', a,'\n\nb = ', b,'\n\np(Z(t=T, u=0:10)|I) = \n', np.exp(p_z_i[:3,-1]))
    # save parameters
    threshold = '2000'
    alpha = str(prior_const)
    run = str(r+1)    
    # np.save(path + 'pi_K_' + str(K) + '_threshold_' + threshold + '_alpha_' + alpha + '_run_' + run, pi)
    # np.save(path + 'mA_K_' + str(K) + '_threshold_' + threshold + '_alpha_' + alpha + '_run_' + run, A)
    # np.save(path + 'va_K_' + str(K) + '_threshold_' + threshold + '_alpha_' + alpha + '_run_' + run, a)
    # np.save(path + 'b_K_' + str(K) + '_threshold_' + threshold + '_alpha_' + alpha + '_run_' + run, b)
    # np.save(path + 'user_class_K_' + str(K) + '_threshold_' + threshold + '_alpha_' + alpha + '_run_' + run, p_z_i[:,-1])

    # number of items to recommend
    num_items = 5000

    # log prob of user each latent class in next period assuming user in Z(t) with log p(Z(t)|I(1:T))
    # result is multiplying transitional prob to prob of user in each latent class at time t
    p_z = logdotexp(p_z_i[:,-1], A)

    # calculate probability that item i is not read in the next time period
    p_noti_z = np.power(1 + b[...,np.newaxis] * np.exp(theta), -a[...,np.newaxis])

    # calculate rank score of the items likely to appear in next time period
    rank_score = -np.exp(p_z) @ p_noti_z

    # generate indices of top num_items to recommend which will be unsorted
    rec_list = np.argpartition(rank_score, -num_items, axis=-1)[:,-num_items:]

    # sort indices by rank score
    rec_list_score = np.array([row[rec_list[i,:]] for i, row in enumerate(rank_score)]) # get the scores of items in rec_list
    sorted_rec_list = np.array([row[np.flip(np.argsort(rec_list_score[i]))] for i, row in enumerate(rec_list)]) # sort the rec_list based on the score

    # check if item in user history
    user_history = np.array([row[:,sorted_rec_list[i]] for i, row in enumerate(users_ds)]) # get all values in user_ds corresponding to the item in rec_list for each user in each time period
    user_history = np.sum(user_history, axis=1) # get boolean array indicating whether each item in sorted_rec_list is in user history (assumes user only has each item at most once)
    if user_history.max() > 1: print('There are repeated ratings of a movie by at least one user')
    # print(user_history.shape)

    # filter sorted_rec_list for items not in user history
    filtered_rec_list = [row[np.logical_not(user_history[i])] for i, row in enumerate(sorted_rec_list)] # each user's list will not have the same amount of items as it depends on user history
    
    # get multi-hot encoding of top N recommended movies for the next period
    mlb = MultiLabelBinarizer(range(N), sparse_output=False) # prediction done on based on one hot encoding indexing i.e. starting index is 0
    top_5_list = [mlb.fit_transform([user[:5]]) for user in filtered_rec_list] # convert top 5 list to one hot encoding
    top_10_list = [mlb.fit_transform([user[:10]]) for user in filtered_rec_list] 

    # test how many of top N recommended movies appear in user's rated list of movies in the test period
    positive_top_5 = [np.multiply(test_ds[i], rec_user) for i, rec_user in enumerate(top_5_list)] # get (#users,#items) boolean vectors indicating whether recommended movie was rating in test period
    users_result_top_5 = [row.sum() for row in positive_top_5] # get list of positive matches per user
    all_result_top_5 = np.sum(users_result_top_5) # total number of positive matches across all users

    positive_top_10 = [np.multiply(test_ds[i], rec_user) for i, rec_user in enumerate(top_10_list)] 
    users_result_top_10 = [row.sum() for row in positive_top_10] # get list of positive matches per user
    all_result_top_10 = np.sum(users_result_top_10) # total number of positive matches across all users

    # total number of movies rated in test period
    test_num_movies_rated = np.sum(test_ds).sum()

    # output results to excel via pandas df
    dict_result = {'# Ratings Threshold':threshold, '# Users':U, '# Movies':N, 'K':K, 'T': T, 't_predict':t_predict,
                'Dirichlet Prior Parameter':alpha, 'Run':run, 'Convergence epsilon': epsilon,
                'Log Likelihood':old_likelihood, 'Iterations':iteration,'Time (min)':run_time*24*60,
                'Avg Time per iteration (s)':run_time*24*60*60/iteration, 
                '# movies rated in test period': test_num_movies_rated, 
                'Total +ve for top 5':all_result_top_5, 
                'Precision of top 5':all_result_top_5/(5*U),
                'Recall of top 5':all_result_top_5/test_num_movies_rated,
                'Total +ve for top 10':all_result_top_10,
                'Precision of top 10':all_result_top_10/(10*U),
                'Recall of top 10':all_result_top_10/test_num_movies_rated
                }
    df_result = pd.DataFrame(data=dict_result, index=[0])
    print(df_result)

    # save result in excel file
    # df_table = pd.read_excel(table_path)
    # df_table = pd.concat([df_table ,df_result], ignore_index=True)
    # df_table.to_excel(table_path, index=False)

Iteration 0 : log likelihood = -19495519.206469715
Iteration 1 : log likelihood = -14457876.151509166
Iteration 2 : log likelihood = -14141379.595077662


KeyboardInterrupt: ignored